# [generated functions](https://docs.julialang.org/en/v1/manual/metaprogramming/#Generated-functions-1)

### What is "generated functions"

1. Generate specialized code depending on the _**types of their arguments**_.
    - _**[Types in JPL](https://docs.julialang.org/en/v1.0/manual/types/) has many unique characteristics, this may enable many potentials**_.
1. A generated function declaration **returns a quoted expression** which then forms the body for the method corresponding to the types of the arguments.
    - _**[Quoting](https://docs.julialang.org/en/v1.0/devdocs/ast/#Quote-expressions-1)**_: create an expression object without using explicit `Expr` constructor.
1. When a `generated function` is called, the expression it returns is compiled and then run.
    - To make this efficient, the compiled result is usually cached.
    - To make this inferable, only a limited subset of the language is usable.

### Why "generated functions"

>**Generated functions provide a flexible way to move work from run time to compile time, at the expense of greater restrictions on allowed constructs.**

`Generated functions` provide a way to implement specialized codes with _**more flexibility**_ and/or _**less code**_ than what can be achieved with multiple dispatch.

- `macros` work with expressions **at parse time** and cannot access the types of their inputs
- a `generated function` gets expanded at a time when the **types of the arguments are known**, but the **function is not yet compiled**.

In [1]:
@generated function foo(x)
    Core.println(x)  # when this line is executed, x is the type of the input
    # return :(x * x)
    quote
        Core.println(x)  # NOTE: This should not be used in real world codes.
                         # Generated functions should not have side effects.
        x * x
    end
end

foo (generic function with 1 method)

1. In the body of the generated function `x` is **the type of the passed argument**.
1. The value returned by the generated function, is the result of **evaluating the quoted expression** we returned from the definition.

Let's see the below example:

In [2]:
x = foo(2)  # Int64 is printed by Core.println
            # 4 is the result of evaluating the returned expression.

Int64
2


4

In [3]:
x  # returned value of a generated function is the result of evaluating the quoted expression.

4

In [4]:
# Let's see another example.
y = foo("hello world ")

String
hello world 


"hello world hello world "

**Evaluate `foo` again with a type that we have already used**!

In [5]:
foo(4)  # There is no printout of "Int64"
        # The body of the generated function was only executed once here for the specific set of argument types, and the result was cached.
        # Expression returned from the generated function on the first invocation was re-used as the method body.

4


16

>the power of a generated function lies in its ability to compute different quoted expressions depending on the types passed to it.

### Principle of generated function

- A correct @generated function **must not** observe any mutable state or cause any mutation of global state.
- Should never write a generated function with [side effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)).

_**In the code snippet below, the parameter `N` is interesting. See the document for [Value types](https://docs.julialang.org/en/v1.0/manual/types/#%22Value-types%22-1) for the reference.**_

In [11]:
function sub2ind_loop(dims::NTuple{N}, I::Integer...) where N
    ind = I[N] - 1
    for i = N - 1 : -1 : 1
        ind = I[i] - 1 + dims[i] * ind
    end
    return ind + 1
end

sub2ind_loop((7, 5), 2, 4)

23

- All the information needed for the loop is embedded in the type information of the arguments.
- It is possible to utilize generated functions to move the iteration to compile-time.

In [12]:
@generated function sub2ind_gen(dims::NTuple{N}, I::Integer...) where N
    ex = :(I[$N] - 1)
    for i = (N - 1) : -1 :1
        ex = :(I[$i] - 1 + dims[$i] * $ex)
    end
    return :($ex + 1)
end

sub2ind_gen (generic function with 1 method)

**What code will this generated function generates? An easy way to find out is to extract its body into another (regular) function:**

In [24]:
function sub2ind_gen_impl(dims::Type{T}, I...) where T <: NTuple{N, Any} where N
    @show T
    @show N

    length(I) == N || return :(error("partial indexing is unsupported"))
    ex = :(I[$N] - 1)
    for i = (N - 1):-1:1
        ex = :(I[$i] - 1 + dims[$i] * $ex)
    end
    return :($ex + 1)
end

@show ans

ans = :(((I[1] - 1) + dims[1] * ((I[2] - 1) + dims[2] * ((I[3] - 1) + dims[3] * (I[4] - 1)))) + 1)


:(((I[1] - 1) + dims[1] * ((I[2] - 1) + dims[2] * ((I[3] - 1) + dims[3] * (I[4] - 1)))) + 1)

In [25]:
@generated function sub2ind_gen(dims::NTuple{N}, I::Integer...) where N
    return sub2ind_gen_impl(dims, I...)
end

sub2ind_gen_impl(Tuple{Int, Int, Int, Int}, Int, Int, Int, Int)

T = NTuple{4,Int64}
N = 4


:(((I[1] - 1) + dims[1] * ((I[2] - 1) + dims[2] * ((I[3] - 1) + dims[3] * (I[4] - 1)))) + 1)